<a href="https://colab.research.google.com/github/starirene9/DeepLearningAssignment/blob/main/Yolo_obj_detection_K2024504_%EA%B5%AC%EB%B9%9B%EB%82%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/ultralytics/ultralytics.git@main
!pip install gradio

# base model download
!wget https://github.com/akanametov/yolo-face/releases/download/v0.0.0/yolov11n-face.pt
!wget https://github.com/akanametov/yolo-face/releases/download/v0.0.0/yolov11s-face.pt

# font download
!apt-get install -y fonts-dejavu

  Cloning https://github.com/ultralytics/ultralytics.git (to revision main) to /tmp/pip-req-build-fwbos97f
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/ultralytics.git /tmp/pip-req-build-fwbos97f
  Resolved https://github.com/ultralytics/ultralytics.git to commit 560c0af14649e79b8592d6d57ca08eb5db00ac23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import gradio as gr
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont, ImageOps

# ── 설정 ─────────────────────────────────────────────────────────────
MODEL_PATH   = "/content/yolov11n-face.pt"
SOURCE_PATH  = "/content/IMG_4866.png"
FONT_PATH    = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
FONT_SIZE    = 40
DEFAULT_CONF = 0.25
DEFAULT_IOU  = 0.45

# ── 리소스 로드 ───────────────────────────────────────────────────────
model = YOLO(MODEL_PATH)
font  = ImageFont.truetype(FONT_PATH, FONT_SIZE)
raw_img = Image.open(SOURCE_PATH)
src_img = ImageOps.exif_transpose(raw_img).convert("RGB")

# ── 얼굴 탐지 함수 ────────────────────────────────────────────────────
def predict_faces(img: Image.Image, conf: float, iou: float) -> Image.Image:
    res  = model.predict(img, conf=conf, iou=iou, imgsz=max(img.size))[0]
    draw = ImageDraw.Draw(img)
    for i, box in enumerate(res.boxes.xyxy.tolist()):
        x1, y1, x2, y2 = map(int, box)
        draw.rectangle([x1, y1, x2, y2], outline="red", width=5)
        draw.text((x1 + 4, y1 + 4), str(i + 1), fill="white", font=font)
    return img

# 최초 결과 한 번 계산
initial_result = predict_faces(src_img.copy(), DEFAULT_CONF, DEFAULT_IOU)

# ── Gradio UI ────────────────────────────────────────────────────────
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 얼굴 탐지 애플리케이션 (자동 결과 표시)")
    gr.Markdown(f"소스 이미지: `{SOURCE_PATH}`")

    #원본 이미지 preview
    image_preview = gr.Image(value=src_img, label="Source Image", interactive=False)

    with gr.Row():
        with gr.Column():
            conf_slider = gr.Slider(
                0, 1, value=DEFAULT_CONF,
                label="Confidence Threshold"
            )
            gr.Markdown("🔍 이 확률 이상이면 얼굴로 인식됩니다. (값이 높을수록 확실한 얼굴만 잡혀요)")

        with gr.Column():
            iou_slider = gr.Slider(
                0, 1, value=DEFAULT_IOU,
                label="IoU Threshold"
            )
            gr.Markdown("📦 겹치는 박스를 하나로 합칠 기준입니다. (값이 클수록 많이 겹쳐야 합쳐져요)")

    detect_btn    = gr.Button("Detect Faces (재실행)")
    result_image  = gr.Image(value=initial_result, type="pil", label="Result")

    # ── 콜백 : 두 단계로 결과 갱신 ────────────────────────────────────
    def detect_cb(conf, iou):
        # 1) 기존 결과를 즉시 지움
        yield gr.Image(value=None)
        # 2) 새 결과 계산 후 출력
        new_img = predict_faces(src_img.copy(), conf, iou)
        yield gr.update(value=new_img)

        #yield 여러 번 중간 결과를 보여줄 수 있음

    detect_btn.click(
        fn=detect_cb,
        inputs=[conf_slider, iou_slider],
        outputs=result_image,
    )

if __name__ == "__main__":
    demo.launch(debug=True)



0: 4032x3040 28 faces, 148.2ms
Speed: 60.4ms preprocess, 148.2ms inference, 1.5ms postprocess per image at shape (1, 3, 4032, 3040)
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dd5b852ab1377c467e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
